In [1]:
import math
import numpy as np
import pandas as pd
import gc
import time

In [2]:
def Quartile_function(four_range,pre_25,F):#四分区域，返回结果为预测的25个点
    F = np.array(F)
    pre_25 = np.array(pre_25)
    pre_res25  = pre_IDW(pre_25,F)
    #print(len(pre_res25))
    return pre_res25  
#[107,27]
def range_pcik4(F):#拾取四分之一的四角范围值
    r1_max = np.max(F[:,0])#107 A
    r1_min = np.min(F[:,0])#C
    r2_max = np.max(F[:,1])#27B
    r2_min = np.min(F[:,1])#D
    four_range = [r1_max,r2_max,r1_min,r2_min]
    return four_range
def Coordinate_pick25(four_range):#计算要求的25个值
    A,B,C,D = four_range[0],four_range[1],four_range[2],four_range[3]
    pre_25_01=[[C,B],[C+(A-C)/4,B],[C+(A-C)/2,B],[C+(A-C)*3/4,B],[A,B]]
    pre_25_02=[[C,D+3/4*(B-D)],[C+(A-C)/4,D+3/4*(B-D)],[C+(A-C)/2,D+3/4*(B-D)],[C+(A-C)*3/4,D+3/4*(B-D)],[A,D+3/4*(B-D)]]
    pre_25_03=[[C,D+1/2*(B-D)],[C+(A-C)/4,D+1/2*(B-D)],[C+(A-C)/2,D+1/2*(B-D)],[C+(A-C)*3/4,D+1/2*(B-D)],[A,D+1/2*(B-D)]]
    pre_25_04=[[C,D+1/4*(B-D)],[C+(A-C)/4,D+1/4*(B-D)],[C+(A-C)/2,D+1/4*(B-D)],[C+(A-C)*3/4,D+1/4*(B-D)],[A,D+1/4*(B-D)]]
    pre_25_05=[[C,D],[C+(A-C)/4,D],[C+(A-C)/2,D],[C+(A-C)*3/4,D],[A,D]]
    pre_25 = pre_25_01+pre_25_02+pre_25_03+pre_25_04+pre_25_05
    #print(pre_25)
    return pre_25
def Trueval(TR,four_range):#筛选四角范围内真实值（已知点）
    F = list()
    A,B,C,D = four_range[0],four_range[1],four_range[2],four_range[3]
    for i in range(len(TR[:,0])):
        x,y = TR[i][0],TR[i][1]
        if C<=x<=A and D<=y<=B:
            tx = [TR[i][0],TR[i][1],TR[i][2]]#需根据数据维数添加参数
            F.append(tx)
    return F
def Pre_cropval(TR,four_range):#筛选四角范围内crop预测点
    Fx = list()
    A,B,C,D = four_range[0],four_range[1],four_range[2],four_range[3]
    for i in range(len(TR[:,0])):
        x,y = TR[i][0],TR[i][1]
        if C<x<A and D<y<B:
            tx = TR[i]#需根据数据维数添加参数
            Fx.append(tx)
    return Fx
def Quar_pick4arr(four_range):#将上一级格网计算结果拆分成四份进行下一级
    A,B,C,D = four_range[0],four_range[1],four_range[2],four_range[3]
    arr1 = [[C,B],[C+(A-C)/2,B],[C,D+1/2*(B-D)],[C+(A-C)/2,D+1/2*(B-D)]]
    arr2 = [[C+(A-C)/2,B],[A,B],[C+(A-C)/2,D+1/2*(B-D)],[A,D+1/2*(B-D)]]
    arr3 = [[C,D+1/2*(B-D)],[C+(A-C)/2,D+1/2*(B-D)],[C,D],[C+(A-C)/2,D]]
    arr4 = [[C+(A-C)/2,D+1/2*(B-D)],[A,D+1/2*(B-D)],[C+(A-C)/2,D],[A,D]]
    res_arr = [arr1,arr2,arr3,arr4]
    res_arr = np.array(res_arr)
    return res_arr # four_range = range_pcik4(res_arr[i])

In [3]:
#更换求距离的函数
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):#经纬度转欧氏距离
    R =  6372.8
    dLon = radians(lon2 - lon1)
    dLat = radians(lat2 - lat1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    d = R * c
    return d
def IDW(x, y, z, xi, yi):#进行反距离权重插值
    lstxyzi = []
    for p in range(len(xi)):
        lstdist = []
        for s in range(len(x)):
            d = (haversine(x[s], y[s], xi[p], yi[p]))
            if d == 0:
                d = 1
            lstdist.append(d)
        sumsup = list((1 / np.power(lstdist, 2)))
        suminf = np.sum(sumsup)
        sumsup = np.sum(np.array(sumsup) * np.array(z))
        u = sumsup / suminf
        xyzi = [xi[p], yi[p], u]
        lstxyzi.append(xyzi)
    return(lstxyzi)
def pre_IDW(pre_25,F):#插值结果函数
    know_lon,know_lat,know_z = F[:,0],F[:,1],F[:,2]
    grid_lon_list = pre_25[:,0].tolist()
    grid_lat_list = pre_25[:,1].tolist()
    res_idw = IDW(know_lon,know_lat,know_z,grid_lon_list,grid_lat_list)
    return res_idw

In [11]:
def sta_flnum(F):
    r1_max = np.max(F[:,0])#107 A
    r1_min = np.min(F[:,0])#C
    r2_max = np.max(F[:,1])#27B
    r2_min = np.min(F[:,1])#D
    #print(r1_max,r1_min,r2_max,r2_min)
    le1 = r1_max - r1_min
    le2 = r2_max - r2_min
    #print(le1,le2)
    m = [8,16,32,64,4]
    for i in range(len(m)):
        num1 = np.ceil(le2 / (0.0001*m[i]))
        num2 = np.ceil(le1 / (0.0001*m[i]))
        num = num1*num2
        if num > 4:
            lev = math.log(m[i],2)
            return num1,num2,num,lev,r1_min,r2_min
def hight_lev(range_File):#最大级别四点范围数组，range_File分组预测数据
    pre_net = sta_flnum(range_File)
    four_range = list()
    a = pow(2,pre_net[3])*0.0001
    for i in range(int(pre_net[0])):
        for j in range(int(pre_net[1])):
            four_point = [pre_net[4]+a*(j+1),pre_net[5]+a*(i+1),pre_net[4]+a*j,pre_net[5]+a*i]
            four_range.append(four_point)
    return four_range    
def floor_function(floor_level,four_range,TR):#floor_level层级 four_range最大四点范围数组 TR已知数据 反池化卷积结构主体函数 
    pre_25 = Coordinate_pick25(four_range)#获取预测方阵点1
    F = Trueval(TR,four_range)#筛选该范围内的真实已知点数据1
    pre_res25 = Quartile_function(four_range,pre_25,F)#预测结果25点
    #np.savetxt("f.txt",pre_res25)
    #Pre_cropval = Pre_cropval(TR,four_range)#最终预测的数据立方体数据整合集1
    F = np.vstack((F,pre_res25))#将预测结果加入到已知点中（可能有重复）1
    res_arr = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分
    if  floor_level == 2 or floor_level == 3 or floor_level == 4 or floor_level == 5 or floor_level == 6:
        i,res_arr_tal5,FM5 =0,list(),list()
        for i in range(len(res_arr)):#4
            four_range = range_pcik4(res_arr[i])
            #print(res_arr[i])
            pre_25 = Coordinate_pick25(four_range)#获取预测方阵点2
            #print(four_range)
            F0 = Trueval(F,four_range)#筛选该范围内的真实已知点数据2
            #print(len(F0))
            pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果2
            F0 = np.vstack((F,pre_res25))#将预测结果加入到已知点中（可能有重复）
            res_arr5 = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分2
            if len(res_arr_tal5) ==0:
                res_arr_tal5 = res_arr5
            elif len(res_arr_tal5) !=0:
                res_arr_tal5=np.vstack((res_arr_tal5,res_arr5))
            if len(FM5) == 0 :
                FM5 = F0
            elif len(FM5) !=0: 
                FM5 = np.vstack((FM5,F0))
        res_arr_tal0 = res_arr_tal5
        FMn = FM5
    if floor_level == 3 or floor_level == 4 or floor_level == 5 or floor_level == 6:
        i,res_arr_tal4,FM4 =0,list(),list()
        for i in range(len(res_arr_tal5)):#16
            four_range = range_pcik4(res_arr_tal5[i])
            pre_25 = Coordinate_pick25(four_range)#获取预测方阵点3
            F0 = Trueval(FM5,four_range)#筛选该范围内的真实已知点数据3   
            #print(len(F0))
            pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果3
            F0 = np.vstack((FM5,pre_res25))#将预测结果加入到已知点中（可能有重复）
            res_arr4 = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分3
            if len(res_arr_tal4) ==0:
                res_arr_tal4 = res_arr4
            elif len(res_arr_tal4) !=0:
                res_arr_tal4=np.vstack((res_arr_tal4,res_arr4))
            if len(FM4) == 0 :
                FM4 = F0
            elif len(FM4) !=0: 
                FM4 = np.vstack((FM4,F0))
        res_arr_tal0 = res_arr_tal4
        FMn = FM4
    if floor_level == 4 or floor_level == 5 or floor_level == 6:
        i,res_arr_tal3,FM3 =0,list(),list()
        for i in range(len(res_arr_tal4)):#64
            four_range = range_pcik4(res_arr_tal4[i])
            pre_25 = Coordinate_pick25(four_range)#获取预测方阵点4
            F0 = Trueval(FM4,four_range)#筛选该范围内的真实已知点数据4
            #print(len(F0))
            pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果4
            F0 = np.vstack((FM4,pre_res25))#将预测结果加入到已知点中（可能有重复）
            res_arr3 = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分4
            if len(res_arr_tal3) ==0:
                res_arr_tal3 = res_arr3
            elif len(res_arr_tal3) !=0:
                res_arr_tal3=np.vstack((res_arr_tal3,res_arr3))
            if len(FM3) == 0 :
                FM3 = F0
            elif len(FM3) !=0: 
                FM3 = np.vstack((FM3,F0)) 
        res_arr_tal0 = res_arr_tal3
        FMn = FM3
    if floor_level == 5 or floor_level == 6:
        i,res_arr_tal2,FM2 =0,list(),list()
        for i in range(len(res_arr_tal3)):#256
            four_range = range_pcik4(res_arr_tal3[i])
            pre_25 = Coordinate_pick25(four_range)#获取预测方阵点5
            F0 = Trueval(FM3,four_range)#筛选该范围内的真实已知点数据5
            #print(len(F0))
            pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果
            F0 = np.vstack((FM3,pre_res25))#将预测结果加入到已知点中（可能有重复）5
            res_arr2 = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分5
            if len(res_arr_tal2) ==0:
                res_arr_tal2 = res_arr2
            elif len(res_arr_tal2) !=0:
                res_arr_tal2=np.vstack((res_arr_tal2,res_arr2))
            if len(FM2) == 0 :
                FM2 = F0
            elif len(FM2) !=0: 
                FM2 = np.vstack((FM2,F0))
        res_arr_tal0 = res_arr_tal2
        FMn = FM2
    if floor_level == 6:
        i,res_arr_tal1,FM1 =0,list(),list()
        for i in range(len(res_arr_tal2)):#1024
            four_range = range_pcik4(res_arr_tal2[i])
            pre_25 = Coordinate_pick25(four_range)#获取预测方阵点6
            F0 = Trueval(FM2,four_range)#筛选该范围内的真实已知点数据6
            #print(len(F0))
            pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果6
            F0 = np.vstack((FM2,pre_res25))#将预测结果加入到已知点中（可能有重复）6
            res_arr1 = Quar_pick4arr(four_range)#拆分函数，将该方阵分成四部分6
            if len(res_arr_tal1) ==0:
                res_arr_tal1 = res_arr1
            elif len(res_arr_tal1) !=0:
                res_arr_tal1=np.vstack((res_arr_tal1,res_arr1))
            if len(FM1) == 0 :
                FM1 = F0
            elif len(FM1) !=0: 
                FM1 = np.vstack((FM1,F0))
        res_arr_tal0 = res_arr_tal1
        FMn = FM1
    i,FM0 =0,list()
    for i in range(len(res_arr_tal0)):#4096
        four_range = range_pcik4(res_arr_tal0[i])
        pre_25 = Coordinate_pick25(four_range)#获取预测方阵点7
        F0 = Trueval(FMn,four_range)#筛选该范围内的真实已知点数据7
        pre_res25 = Quartile_function(four_range,pre_25,F0)#预测25点结果7
        F0 = np.vstack((FMn,pre_res25))#将预测结果加入到已知点中（可能有重复）7
        if len(FM0) == 0 :
            FM0 = F0
        elif len(FM0) !=0: 
            FM0 = np.vstack((FM0,F0)) 
    return FM0

In [5]:
def haveraster_data(filename,Separator):
    N=list()
    f = open(filename,'r') 
    line = f.readline() 
    while line:
        c,d,e,g = line.split(Separator)
        x = eval(c)  #X坐标
        y = eval(d)  #Y坐标
        z = eval(e)  #Z坐标
        n = eval(g)
        M = [x,y,z,n]
        N.append(M)
        line = f.readline() 
    f.close()
    return N
def prepoint_data(filename,Separator):
    N=list()
    f = open(filename,'r') 
    line = f.readline() 
    while line:
        c,d,e,g = line.split(Separator)
        x = eval(c)  #X坐标
        y = eval(d)  #Y坐标
        z = eval(e)  #Z坐标
        w = eval(g)
        M = [x,y,z,w]
        N.append(M)
        line = f.readline() 
    f.close()
    return N
def circulate(Pr,Kn,num):
    Pred,Know = list(),list()
    for i in range(len(Pr)):
        if Pr[i][3] == num:
            arr99 = [Pr[i][0],Pr[i][1],Pr[i][2],Pr[i][3]]
            #arr = [Pr[i][0],Pr[i][1],Pr[i][2],Pr[i][3]]
            Pred.append(arr99)
    for j in range(len(Kn)):
        if Kn[j][3] == num:
            arr0 = [Kn[j][0],Kn[j][1],Kn[j][2],Kn[j][3]]
            Know.append(arr0)
    return Pred,Know

In [6]:
#检索函数
def data_merge(pre_value,fect_value):
    respf_value  = list()
    for i in range(len(pre_value)):
        x,y = pre_value[i][0],pre_value[i][1]
        j = 0
        for j in range(len(fect_value)):
            a,b = fect_value[j][0],fect_value[j][1]
            if pow(a - x,2) + pow(b-y,2) < pow(0.00005,2):
                pf_value = [x,y,pre_value[i][2],pre_value[i][3],fect_value[j][2]]
                respf_value.append(pf_value)                
                break
    return respf_value
def output_data(arr):
    arr = np.array(arr)
    for i in range(len(arr)):
        s= open("分区2-25_预测结果.txt", 'a+')
        s.write(f'{arr[i][0]},{arr[i][1]},{arr[i][2]},{arr[i][3]},{arr[i][4]}\n')
    #print("数据导出完成！")    

In [7]:
#运行函数
file_pre = r"C:\\Users\\穆田宝\\Desktop\\大论文数据\\源数据\\crop_point.txt"
file_dem = r"C:\\Users\\穆田宝\\Desktop\\大论文数据\\源数据\\DEM_point.txt"
prepoint = prepoint_data(file_pre,",")
havepoint =haveraster_data(file_dem,",")

In [ ]:
j = 2
while j <= 25:
    Pred,Know = circulate(prepoint,havepoint,j)
    Pred = np.array(Pred)
    Know = np.array(Know)
    floor_level = sta_flnum(Pred)[3]
    print(sta_flnum(Pred))
    four_range_all = hight_lev(Pred)    
    start_time = time.time()    # 程序开始时间
    for i in range(len(four_range_all)):
        four_range =four_range_all[i]
        Vali_test = Trueval(Know,four_range)
        if len(Vali_test) > 0:
            dat_a = Pre_cropval(Pred,four_range) 
            FM = floor_function(floor_level,four_range,Know)
            if i % 1000 == 0:
                print("第{0}个网格已完成！".format(i+1))
            arr = data_merge(dat_a,FM)
            output_data(arr)
            gc.collect()#清空当前循环的缓存            
    end_time = time.time()    # 程序结束时间
    run_time = end_time - start_time    # 程序的运行时间，单位为秒
    print("{0}用时：{1}".format(j,run_time))
    j = j+1

(45.0, 48.0, 2160.0, 3.0, 109.11530303955078, 27.61290740966797)
第1001个网格已完成！
第2001个网格已完成！
2用时：1318.176346540451
(42.0, 43.0, 1806.0, 3.0, 109.14234161376953, 27.60625648498535)
3用时：1154.3293061256409
(46.0, 34.0, 1564.0, 3.0, 109.17227935791016, 27.605398178100586)
第1个网格已完成！
第1001个网格已完成！
4用时：798.0323634147644
(78.0, 112.0, 8736.0, 3.0, 108.89557647705078, 27.491456985473633)
第3001个网格已完成！
5用时：2614.961009979248
(84.0, 130.0, 10920.0, 3.0, 108.8431167602539, 27.49403190612793)
第2001个网格已完成！
第3001个网格已完成！
第5001个网格已完成！
第6001个网格已完成！
第8001个网格已完成！
第10001个网格已完成！
6用时：6680.748314619064
(44.0, 66.0, 2904.0, 3.0, 108.86982727050781, 27.54949951171875)
第1001个网格已完成！
第2001个网格已完成！
7用时：1920.5562810897827
(88.0, 140.0, 12320.0, 3.0, 108.9393539428711, 27.44510841369629)
第3001个网格已完成！
第4001个网格已完成！
第5001个网格已完成！
第6001个网格已完成！
第9001个网格已完成！
第10001个网格已完成！
8用时：8334.085030555725
(63.0, 138.0, 8694.0, 3.0, 109.2037124633789, 27.562374114990234)
第1个网格已完成！
第1001个网格已完成！
第6001个网格已完成！
第7001个网格已完成！
第8001个网格已完成！
9用时：2931.5

In [ ]:
import os
os.system('shutdown -s -f -t 60')